# Stock price prediction: A machine learning approach

Purpose: Predict stock price using Machine Learning methods, such as Support Vector Machines, Deep Neural Networks and Random Forest \
Authors: Caio Lopes De Souza, Silvio Sandoval Zocchi, Gabriel Rodrigues Palma

# Packages used in the project

In [115]:
# Data manipulation modules
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

# Data visualisation modules
import matplotlib.pyplot as plt

# Machine Learning modules
from sklearn.ensemble import RandomForestClassifier

# Cross-validation modules 
from sklearn.model_selection import TimeSeriesSplit

# Grid search module
from sklearn.model_selection import GridSearchCV

# Evaluation metrics modules
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [116]:
def get_train_test_data(cripto_series, test_percentage):
    '''This functions creates the training and test data for the stock time series'''
    time_series_length = len(cripto_series)
    test_series = cripto_series.iloc[int(1 - time_series_length*test_percentage):]
    train_series = cripto_series.iloc[:int(1 - time_series_length*test_percentage)]
    
    return(test_series, train_series)

def get_rate_of_change(cripto_series, variable, n_days):
    '''This function computes several values for the rate of change and add them into a dataset'''
    for day in n_days: 
        variable_name = f"{day}_days_of_change"
        cripto_series[variable_name] = (cripto_series[variable] - cripto_series[variable].shift(day)) / cripto_series[variable].shift(day)
        
def get_moving_average(cripto_series, variable, n_days):
    '''This function computes several values for the moving average and add them into a dataset'''
    for day in n_days:
        variable_name = f"{day}_day_moving_average"
        cripto_series[variable_name] = cripto_series[variable].rolling(day).mean()
        
def get_ratio(cripto_series, variable_1, variable_2):
    '''This function computes the ratio between two variables and add it into a dataset'''
    variable_name = f"{variable_1}_{variable_2}_ratio"
    cripto_series[variable_name] = cripto_series[variable_1] / cripto_series[variable_2]

## Reading the Data

In [117]:
cripto_series = pd.read_csv('https://raw.githubusercontent.com/CaioLSouza/stock_price_prediction/main/Data/revolut_crypto_data.csv', index_col='time_open')
bitcoin_series = cripto_series.loc[cripto_series['name'] == 'Bitcoin']

## Cleaning variables

In [118]:
bitcoin_series.drop(['timestamp', 'id', 'Unnamed: 0', 'slug', 'time_close', 'time_high', 'time_low', 'name', 'symbol', 'ref_cur'], axis=1, inplace=True)

In [119]:
bitcoin_series.head()

,open,high,low,close,volume,market_cap
time_open,,,,,,
2013-04-29,134.444000,147.488007,134.000000,144.539993,0.0,1.603769e+09
2013-04-30,144.000000,146.929993,134.050003,139.000000,0.0,1.542813e+09
2013-05-01,139.000000,139.889999,107.720001,116.989998,0.0,1.298955e+09
2013-05-02,116.379997,125.599998,92.281898,105.209999,0.0,1.168517e+09
2013-05-03,106.250000,108.127998,79.099998,97.750000,0.0,1.085995e+09


In [120]:
bitcoin_series['target']  = np.array(bitcoin_series['close'].shift(-1) > bitcoin_series['close']).astype(int)

## Creating new variables

In [121]:
get_rate_of_change(bitcoin_series, 'close', [3, 5, 7, 15])

In [122]:
get_moving_average(bitcoin_series, 'close', [3, 5, 7, 15])

In [123]:
get_ratio(bitcoin_series, 'close', 'open')
get_ratio(bitcoin_series, 'close', 'low')
get_ratio(bitcoin_series, 'close', 'high')

In [124]:
bitcoin_series.isnull().sum()

open                      0
high                      0
low                       0
close                     0
volume                    0
market_cap                0
target                    0
3_days_of_change          3
5_days_of_change          5
7_days_of_change          7
15_days_of_change        15
3_day_moving_average      2
5_day_moving_average      4
7_day_moving_average      6
15_day_moving_average    14
close_open_ratio          0
close_low_ratio           0
close_high_ratio          0
dtype: int64

## Proposed approach

In [125]:
bitcoin_series.dropna(axis=0, inplace=True)

In [126]:
test_series, train_series = get_train_test_data(bitcoin_series,
                                                test_percentage = 0.1)

In [127]:
predictors = ['close', 'open', 'high', 'low', 'volume', '3_days_of_change',
             '5_days_of_change', '7_days_of_change', '15_days_of_change',
             '3_day_moving_average', '5_day_moving_average', '7_day_moving_average', '15_day_moving_average',
             'close_open_ratio', 'close_low_ratio', 'close_high_ratio', 'market_cap']

In [128]:
tscv = TimeSeriesSplit()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500], 'max_depth': [400, 500, 600], 'min_samples_split': [100, 200, 300]}
]

forest_class = RandomForestClassifier()

grid_search = GridSearchCV(forest_class, param_grid, cv=tscv,
                           scoring='precision')

grid_search.fit(train_series[predictors], train_series['target'])

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=RandomForestClassifier(),
             param_grid=[{'max_depth': [400, 500, 600],
                          'min_samples_split': [100, 200, 300],
                          'n_estimators': [100, 200, 300, 400, 500]}],
             scoring='precision')

In [129]:
grid_search.best_params_

{'max_depth': 600, 'min_samples_split': 200, 'n_estimators': 400}

In [ ]:
def train_and_predict(model, train[predictors], train[target], test[predictors]):
    model.fit(train[predictors], train[target])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index)
    return preds

In [130]:
model = RandomForestClassifier(n_estimators=100, min_samples_split = 200, max_depth=500 ,random_state=True)

In [131]:
model.fit(train_series[predictors], train_series['target'])

RandomForestClassifier(max_depth=500, min_samples_split=200, random_state=True)

In [132]:
preds = model.predict(test_series[predictors])
preds = preds.ravel()
preds = pd.Series(preds, index=test_series.index)

In [133]:
precision_score(test_series['target'], preds)

0.510989010989011

In [134]:
recall_score(test_series['target'], preds)

0.5670731707317073

In [ ]:
accuracy_score(test_series['target'], preds)